In [5]:
import arcpy
from arcpy import env
from arcpy.sa import *
from arcgis.gis import GIS
import geopandas as gpd
import matplotlib.pyplot as plt

ImportError: DLL load failed while importing lib: The specified procedure could not be found.

In [3]:
# 设置工作空间
arcpy.env.workspace = "D:/SSCI/StoryMAP/DMaping/DemoData"
env.overwriteOutput = True

#建立地理数据库
gdb_name = "MyNewGeodatabase.gdb"
arcpy.CreateFileGDB_management(env.workspace, gdb_name)

<Result 'D:/SSCI/StoryMAP/DMaping/DemoData\\MyNewGeodatabase.gdb'>

In [4]:
# 加载数据
block_shapefile = "Block.shp"
maui_land_cover_shapefile = "MauiLandCover.shp"

# 设置地图底板
fig, ax = plt.subplots(figsize=(10, 10))

# 绘制数据
block_shapefile.plot(ax=ax, color='blue', alpha=0.5, edgecolor='k', label='Block')
maui_land_cover_shapefile.plot(ax=ax, color='green', alpha=0.5, label='Maui Land Cover')

# 设置标题和图例
plt.title('Block and Maui Land Cover Map')
plt.legend()

# 显示图表
plt.show()

NameError: name 'plt' is not defined

In [ ]:
#添加“RA”字段
arcpy.AddField_management("MauiLandCover.shp", "RA", "SHORT")

land_use_weights = {
    "High Intensity Developed": 46,
    "Open Space Developed": 26,
    "Cultivated Land": 10,
    "Pasture or Hay": 5,
    "Grassland": 4,
    "Evergreen Forest": 3,
    "Scrub Shrub": 3,
    "Palustrine Forested Wetland": 1,
    "Palustrine Scrub Shrub Wetland": 1,
    "Palustrine Emergent Wetland": 1,
    "Unconsolidated Shore": 0,
    "Bare Land": 0,
    "Open Water": 0,
    "Palustrine Aquatic Bed": 0
}

with arcpy.da.UpdateCursor("MauiLandCover.shp", ["LandCoverType", "RA"]) as cursor:
    for row in cursor:
        land_cover_type = row[0]
        if land_cover_type in land_use_weights:
            row[1] = land_use_weights[land_cover_type]
        else:
            row[1] = 0  # 默认值，如果类型不在字典里
        cursor.updateRow(row)

将MauiLandcover中的“RA”字段和“gridcode”字段使用feature to raster工具转换成两个raster

In [ ]:
# 指定输入的 shapefile
input_feature = "MauiLandCover.shp"

# 设置输出栅格的分辨率
cell_size = 20

# 转换 'RA' 字段到栅格
ra_raster_output = "RA_Raster.tif"
arcpy.FeatureToRaster_conversion(input_feature, "RA", ra_raster_output, cell_size)

# 转换 'gridcode' 字段到栅格
gridcode_raster_output = "GridCode_Raster.tif"
arcpy.FeatureToRaster_conversion(input_feature, "gridcode", gridcode_raster_output, cell_size)

将Maui Block中字段“OBJECTID”和“Total Population”字段用feature to raster转换为栅格

In [ ]:
# 输入 Shapefile
input_feature = "Block.shp"

# 设置输出栅格的分辨率
cell_size = 20  # 根据需要调整分辨率

# 转换 'OBJECTID' 字段到栅格
objectid_raster_output = "ObjectID_Raster.tif"
arcpy.FeatureToRaster_conversion(input_feature, "OBJECTID", objectid_raster_output, cell_size)

# 转换 'Total Population' 字段到栅格
total_pop_raster_output = "TotalPopulation_Raster.tif"
arcpy.FeatureToRaster_conversion(input_feature, "Total Population", total_pop_raster_output, cell_size)


In [ ]:
# 处理Land Cover数据并展示
tab_area = TabulateArea("Block_ID_Raster", "Value", "Maui_Landcover", "Value", "Tabulate_Area_Result")
mapview.add_layer(tab_area)  # 添加处理结果到地图

# 计算人口密度并展示
population_density = (Raster("RA") * Raster("Pop_Raster") * 400) / (Raster("Total_Pixel_Raster") * Raster("Expected_Pop_Raster"))
population_density.save("Population_Density_Raster")
mapview.add_layer("Population_Density_Raster")  # 展示人口密度结果